In [1]:
import pandas as pd

In [2]:
da_dict = pd.read_csv(f'F:\\Guvi docs\\DataSpark\\Data_Dictionary.csv')
df_sales = pd.read_csv(f'F:\\Guvi docs\\DataSpark\\Sales.csv')
df_exchange = pd.read_csv(f'F:\\Guvi docs\\DataSpark\\Exchange_Rates.csv')
df_stores = pd.read_csv(f'F:\\Guvi docs\\DataSpark\\Stores.csv')
df_customers = pd.read_csv(f'F:\\Guvi docs\\DataSpark\\Customers.csv',encoding='latin1')
df_products = pd.read_csv(f'F:\\Guvi docs\\DataSpark\\Products.csv')

In [ ]:
'''where(pd.notna(df), None) is simple and efficient if you want a global transformation across the entire DataFrame to replace NaN with None.
applymap is more flexible and allows you to apply custom functions element-wise if you need more control.
fillna() is a quick option if you just want to replace NaN with None for specific columns.'''

In [67]:
import pymysql

def insert_table(df,table):
    
    #connect to the package using connection strings
    host = 'localhost'       # Database host, e.g., 'localhost' or '127.0.0.1'
    user = 'root'   # Your MySQL username
    password = 'He007#W=Got765ked' # Your MySQL password
    database = 'foundation' # The name of the database you want to connect to
    connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

    try:
        cursor = connection.cursor()

        # Dynamically extract column names and values from DataFrame
        columns = df.columns.tolist()  # List of column names
        values = df.values.tolist()    # List of row values as tuples

        # Prepare the insert query dynamically
        insert_query = f"INSERT INTO {table} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"

        # Bulk insert the data
        cursor.executemany(insert_query, values)

        # Commit the changes to the database and close the connection
        connection.commit()
        cursor.close()
        connection.close()

        print("Data inserted successfully!")

    except Exception as e:
        print(f"Error in inserting the data in the table:{e}")

    return 

    

In [ ]:
df_customers['Birthday'] = pd.to_datetime(df_customers['Birthday'])
df_customers['Gender'] = df_customers['Gender'].astype('category')
df_customers['Continent'] = df_customers['Continent'].astype('category')

#rename some columns
df_customers = df_customers.rename(columns={'State Code': 'StateCode', 'Zip Code':'ZipCode'})
df_customers = df_customers.where(pd.notna(df_customers),None)

df_customers.info()
df_customers.duplicated().sum()

#insert into table
insert_table(df_customers,"customers")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CustomerKey  15266 non-null  int64         
 1   Gender       15266 non-null  category      
 2   Name         15266 non-null  object        
 3   City         15266 non-null  object        
 4   StateCode    15256 non-null  object        
 5   State        15266 non-null  object        
 6   ZipCode      15266 non-null  object        
 7   Country      15266 non-null  object        
 8   Continent    15266 non-null  category      
 9   Birthday     15266 non-null  datetime64[ns]
dtypes: category(2), datetime64[ns](1), int64(1), object(6)
memory usage: 984.3+ KB
Data inserted successfully!


#df_sales = df_sales.drop('Delivery Date', axis = 1)
df_sales['Order Date'] = pd.to_datetime(df_sales['Order Date'])
df_sales['Currency Code'] = df_sales['Currency Code'].astype('category')
df_sales = df_sales.where(pd.notna(df_sales),None)
df_sales.info()

In [71]:
df_exchange['Currency'] = df_exchange['Currency'].astype('category')

df_exchange['Date'] = pd.to_datetime(df_exchange['Date'])
df_exchange = df_exchange.where(pd.notna(df_exchange),None)

df_exchange.columns = df_exchange.columns.str.replace(' ', '')
df_exchange.info()

insert_table(df_exchange,"Exchange")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11215 entries, 0 to 11214
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      11215 non-null  datetime64[ns]
 1   Currency  11215 non-null  category      
 2   Exchange  11215 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(1)
memory usage: 186.5 KB
Data inserted successfully!


In [88]:

df_stores['Open Date'] = pd.to_datetime(df_stores['Open Date'])
#df_exchange.columns = df_exchange.columns.str.replace(' ', '')
#df_stores['Square Meters'] = df_stores['Square Meters'].apply(lambda x: None if pd.isna(x) else x) 

df_stores['Square Meters'] = df_stores['Square Meters'].fillna(value = None)
df_stores.info()

print(df_stores)
insert_table(df_stores,"Stores")

ValueError: Must specify a fill 'value' or 'method'.

In [19]:
df_products['Color'] = df_products['Color'].astype('category')
df_products['Unit Cost USD'] = (df_products['Unit Cost USD'].astype(str)).str.replace(r'[^0-9.]', '', regex=True).astype(float)
df_products['Unit Price USD'] = (df_products['Unit Price USD'].astype(str)).str.replace(r'[^0-9.]', '', regex=True).astype(float)
#.str.replace(['$',','],'').astype(float)
df_products.info()

df_products['Color'].unique()
df_products['Unit Cost USD']



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   ProductKey      2517 non-null   int64   
 1   Product Name    2517 non-null   object  
 2   Brand           2517 non-null   object  
 3   Color           2517 non-null   category
 4   Unit Cost USD   2517 non-null   float64 
 5   Unit Price USD  2517 non-null   float64 
 6   SubcategoryKey  2517 non-null   int64   
 7   Subcategory     2517 non-null   object  
 8   CategoryKey     2517 non-null   int64   
 9   Category        2517 non-null   object  
dtypes: category(1), float64(2), int64(3), object(4)
memory usage: 180.2+ KB


0        6.62
1        6.62
2        7.40
3       11.00
4       11.00
        ...  
2512    43.07
2513    43.07
2514     1.71
2515     1.71
2516     1.71
Name: Unit Cost USD, Length: 2517, dtype: float64

In [8]:
#import pymysql package
import pymysql

#connect to the package using connection strings
host = 'localhost'       # Database host, e.g., 'localhost' or '127.0.0.1'
user = 'root'   # Your MySQL username
password = 'He007#W=Got765ked' # Your MySQL password
database = 'foundation' # The name of the database you want to connect to
connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
print("Estabilized connection")
connection.close()

Estabilized connection


In [64]:
import pymysql

def insert_table(df,table):
    
    #connect to the package using connection strings
    host = 'localhost'       # Database host, e.g., 'localhost' or '127.0.0.1'
    user = 'root'   # Your MySQL username
    password = 'He007#W=Got765ked' # Your MySQL password
    database = 'foundation' # The name of the database you want to connect to
    connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

    try:
        cursor = connection.cursor()

        # Dynamically extract column names and values from DataFrame
        columns = df.columns.tolist()  # List of column names
        values = df.values.tolist()    # List of row values as tuples

        # Prepare the insert query dynamically
        insert_query = f"INSERT INTO {table} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"

        # Bulk insert the data
        cursor.executemany(insert_query, values)

        # Commit the changes to the database and close the connection
        connection.commit()
        cursor.close()
        connection.close()

        print("Data inserted successfully!")

    except Exception as e:
        print(f"Error in inserting the data in the table:{e}")

    return 

    

In [65]:
insert_table(df_customers,"customers")

Data inserted successfully!
